Data:

In [1]:
import xml.etree.ElementTree as ET
mytree = ET.parse('/notebook/Touche22/touche2021_val/topics-task2-51-100.xml') # this is unzip of topics-task2-2022.zip
myroot = mytree.getroot()
titles = list()
for item in myroot:
    for x in item:        
        if x.tag == "objects": # Specify the field, e.g., title
            titles.append(x.text)

print(len(titles))
print(titles)

0
[]


In [2]:
def read_xml(filename):
    # convert file filename to list of tuples (number_of_topic, title_of_topic) 
    # input: filename string
    # output: list of corresponding tuples
    answer_list = []
    xmldoc = minidom.parse(filename)
    itemlist = xmldoc.getElementsByTagName('topics')
    print(len(itemlist))
    print(itemlist)
    topic_list = itemlist[0].getElementsByTagName('topic')
    print (len(topic_list))
    for topic in topic_list:
        tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
        answer_list.append(tuple_for_add)
    return answer_list


In [6]:
from xml.dom import minidom
import re

topics_2021 = read_xml('/notebook/Touche22/touche2021_val/topics-task2-51-100.xml')

1
[<DOM Element: topics at 0x7fc3801337d0>]
50


In [7]:
topics_2021

[('51', 'What is better at reducing fever in children, Ibuprofen or Aspirin?'),
 ('52', 'What are the best rice cookers?'),
 ('53', 'Should I buy steel or ceramic knives?'),
 ('54', 'Is morning or afternoon sun the best for fruit trees?'),
 ('55',
  'What is better for back pain, chiropractic therapy or physical therapy?'),
 ('56', 'Is Kenya or Tanzania better for a safari?'),
 ('57', "How is a Master's degree different from a Bachelor's degree?"),
 ('58', 'Which is better, Family Guy or The Simpsons?'),
 ('59', 'Which is more difficult, skiing or snowboarding?'),
 ('60', 'Why is basketball better than football?'),
 ('61', 'Who is stronger, Hulk or Superman?'),
 ('62',
  'Plastic pots or ceramic pots, which is better in terms of plant health?'),
 ('63', 'What is the coldest temperature ever recorded on Earth, and where?'),
 ('64', 'What is the most popular beer in Australia?'),
 ('65', 'Are dogs better than humans?'),
 ('66', 'What diet gives the best result to lose weight?'),
 ('67', 

In [8]:
import csv
with open("/notebook/ColBERT/collections/touche21_queries.tsv", "w") as csv_file:
        writer = csv.writer(csv_file, delimiter='\t')
        for line in topics_2021:
            #print (line)
            #print (type(line[0]))
            line = [str(int(line[0]) - 51), line[1]]
            writer.writerow(line)

In [9]:
tsv_file = open("/notebook/ColBERT/experiments/TOUCHE21_psg/retrieve.py/2022-02-15_17.21.46/ranking.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")
i = 0
for row in read_tsv:
    if (i < 10):
        print(row)
        i = i +1

['0', '18', '1']
['0', '10', '2']
['0', '24', '3']
['0', '4', '4']
['0', '30', '5']
['0', '21', '6']
['0', '55', '7']
['0', '31', '8']
['0', '52', '9']
['0', '50', '10']


In [10]:
import csv
tsv_file = open("/notebook/ColBERT/collections/touche21_queries.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

i = 0
for row in read_tsv:
    if (i < 10):
        print(row)
        i = i +1

['0', 'What is better at reducing fever in children, Ibuprofen or Aspirin?']
['1', 'What are the best rice cookers?']
['2', 'Should I buy steel or ceramic knives?']
['3', 'Is morning or afternoon sun the best for fruit trees?']
['4', 'What is better for back pain, chiropractic therapy or physical therapy?']
['5', 'Is Kenya or Tanzania better for a safari?']
['6', "How is a Master's degree different from a Bachelor's degree?"]
['7', 'Which is better, Family Guy or The Simpsons?']
['8', 'Which is more difficult, skiing or snowboarding?']
['9', 'Why is basketball better than football?']


In [42]:
def run_baseline(output_dir = '/notebook/Touche22/touche2021_val/outs', input_dir = '/notebook/Touche22/touche2021_val/', input_file = 'topics-task2-51-100.xml'):
    
    # load responses for all queries from file 
    with open('list_of_un_answ_2021.pcl', 'rb') as f:
        my_response_list = pickle.load(f)
    #my_response_list = create_list_of_unigue_answers(input_dir = input_dir, input_file = input_file)
    
    print ("len my responses list", len(my_response_list))
    
    print ("input_dir + '/' + input_file", input_dir + input_file)
    list_of_tuples = read_xml(input_dir + input_file)
    common_list = []

    print ("baseline", flush = True)
    for ind_q, elem in enumerate(list_of_tuples):
        qid = elem[0]
        Q0 = 'Q0'
        query = elem[1]
        tag = 'MyBaselineFilterResponse'
        responses = list(zip(*my_response_list[qid]))
        
        scores0 = responses[0]
        docs = responses[1]
        #print ("1")
        titles = responses[2]
        #print ("2")
        answers_bodies = responses[3]
        #print ("3")
        # print (scores0, scores3, scores)
        qids = [qid]*len(scores0)
        Q0s = [Q0 for elem in scores0]
        tags = [tag for elem in scores0]
        part_of_commom_list = list(zip(qids, Q0s, docs, scores0, tags))
        part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 

        qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)

        ranks = range(1, len(scores) + 1)
        part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
        common_list = common_list + part_of_commom_list
        print ("len(common_list)", len(common_list))
    return common_list
    with open('./baseline.qrels', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

In [64]:
with open('/notebook/Touche22/touche2021_val/outs/lines[:10].qrels') as file:
    lines = file.readlines()

In [65]:
lines[:10]

['80 Q0 clueweb12-0000wb-70-11489 1 1.0 colbert\n',
 '80 Q0 clueweb12-0309wb-46-24133 2 0.5 colbert\n',
 '80 Q0 clueweb12-1314wb-92-15027 3 0.3333333333333333 colbert\n',
 '80 Q0 clueweb12-1200wb-79-17799 4 0.25 colbert\n',
 '80 Q0 clueweb12-0105wb-60-22777 5 0.2 colbert\n',
 '80 Q0 clueweb12-1516wb-28-23404 6 0.16666666666666666 colbert\n',
 '80 Q0 clueweb12-0714wb-55-00453 7 0.14285714285714285 colbert\n',
 '80 Q0 clueweb12-1913wb-70-08059 8 0.125 colbert\n',
 '80 Q0 clueweb12-1616wb-48-29292 9 0.1111111111111111 colbert\n',
 '80 Q0 clueweb12-0108wb-96-17069 10 0.1 colbert\n']

In [66]:
with open('/notebook/Touche22/touche2021_val/outs/colbert_90.qrels') as file:
    lines = file.readlines()

In [67]:
lines[:10]

['91 Q0 clueweb12-0700wb-90-19409 1 1.0 colbert\n',
 '91 Q0 clueweb12-0011wb-60-27834 2 0.5 colbert\n',
 '91 Q0 clueweb12-1306wb-49-06042 3 0.3333333333333333 colbert\n',
 '91 Q0 clueweb12-0812wb-61-14830 4 0.25 colbert\n',
 '91 Q0 clueweb12-0810wb-00-06776 5 0.2 colbert\n',
 '91 Q0 clueweb12-1006wb-58-12637 6 0.16666666666666666 colbert\n',
 '91 Q0 clueweb12-1303wb-58-05620 7 0.14285714285714285 colbert\n',
 '91 Q0 clueweb12-0500wb-08-20545 8 0.125 colbert\n',
 '91 Q0 clueweb12-1413wb-94-04951 9 0.1111111111111111 colbert\n',
 '91 Q0 clueweb12-0916wb-34-13384 10 0.1 colbert\n']

In [18]:
import pickle
with open('/notebook/Touche22/touche2021_val/list_of_un_answ_2021.pcl', 'rb') as f:
        my_response_list = pickle.load(f)

In [20]:
with open('/notebook/Touche22/touche2021_val/list_of_un_answ_2021.pcl', 'rb') as f:
        my_response_list = pickle.load(f)

big_dict = {}
for key in my_response_list.keys():
    for elem in my_response_list[key]:
        big_dict.update({elem[1]:elem[3]})
        
len(big_dict)    

4905

In [22]:
import html
html.unescape(strr)

NameError: name 'strr' is not defined

In [23]:
import html
list_of_passages1 = []
ind = 0
mapping_dict = {} # {ind : clue_web_index}
for key in my_response_list.keys():
    for elem in my_response_list[key]:
        list_of_passages1.append(html.unescape(elem[3]))
        mapping_dict.update({ind : elem[1]})
        ind += 1
    
    

In [24]:
with open('/notebook/Touche22/touche2021_val/outs/colbert_index_match.pickle', 'wb') as handle:
    pickle.dump(mapping_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [25]:
strs = []
for ind, elem in enumerate(list_of_passages):
    strs.append([str(ind),elem])

NameError: name 'list_of_passages' is not defined

In [190]:
strs[:4]

[['0',
  'additionally, these medications can become toxic, even at correct dosages, among young children suffering from dehydration due to fever, vomiting and/or diarrhea. excess levels of acetaminophen or nsaids are not harmless.'],
 ['1',
  "but is your child's fever really a cause for concern, and is lowering it the best choice? in most cases, a simple fever is actually a sign that your child's immune system is working at its best."],
 ['2',
  'in children, a fever that is equal to or greater than 102.2° should be treated. children older than two months of age with a fever of 102° or higher that does not respond to fever-reducing medication should be seen by a physician.'],
 ['3',
  'it is effective in reducing fever and is also prescribed for the treatment of arthritic conditions such rheumatoid- and osteoarthritis, muscle injury, and acute attacks of gout.']]

In [191]:
import csv
with open("/notebook/ColBERT/collections/touche21_psgs.tsv", "w") as csv_file:
        writer = csv.writer(csv_file, delimiter='\t')
        for line in strs:
            #print (line)
            #print (type(line[0]))
            writer.writerow(line)

In [192]:
import csv
tsv_file = open("/notebook/ColBERT/collections/touche21_psgs.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

i = 0
for row in read_tsv:
    if (i < 10):
        print(row)
        i = i +1

['0', 'additionally, these medications can become toxic, even at correct dosages, among young children suffering from dehydration due to fever, vomiting and/or diarrhea. excess levels of acetaminophen or nsaids are not harmless.']
['1', "but is your child's fever really a cause for concern, and is lowering it the best choice? in most cases, a simple fever is actually a sign that your child's immune system is working at its best."]
['2', 'in children, a fever that is equal to or greater than 102.2° should be treated. children older than two months of age with a fever of 102° or higher that does not respond to fever-reducing medication should be seen by a physician.']
['3', 'it is effective in reducing fever and is also prescribed for the treatment of arthritic conditions such rheumatoid- and osteoarthritis, muscle injury, and acute attacks of gout.']
['4', 'ibuprofen (nurofen and others) is another fever-reducing medication that relieves mild to moderate aches and pains. be sure to take

## colbert

In [69]:
len(mapping_dict)

4912

In [68]:
tsv_file = open("/notebook/Touche22/touche2021_val/outs/ranking_l2_90.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

common_list = []
i = 0
for row in read_tsv:
    if (i < 10):
        print(row)
        i = i +1
    doq_n = str(int(row[0]) + 51)
    pass_idx = mapping_dict[int(row[1])]
    rank = row[2]
    
    res_str = doq_n + ' Q0 ' + str(pass_idx) + " " + str(rank) + " " + str(1./int(rank))+ " colbert"
    
    common_list.append(res_str)

['91', '25865', '1']


KeyError: 25865

In [53]:
tsv_file = open("/notebook/Touche22/touche2021_val/91_100_query/ranking_colbert.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

common_list = []
i = 0
for row in read_tsv:
    if (i < 10):
        print(row)
        i = i +1
    if (int(row[2]) < 101):
        doq_n = str(int(row[0]))
        pass_idx = mapping_dict[int(row[1])]
        rank = row[2]

        res_str = doq_n + ' Q0 ' + str(pass_idx) + " " + str(rank) + " " + str(1./int(rank))+ " colbert"

        common_list.append(res_str)

['91', '3976', '1']
['91', '4003', '2']
['91', '3999', '3']
['91', '4019', '4']
['91', '3965', '5']
['91', '3964', '6']
['91', '4027', '7']
['91', '3978', '8']
['91', '3985', '9']
['91', '3959', '10']


In [33]:
def print_common_list(common_list, numb, num2doc_ind, doc_ind2pass, big_dict):
    ind_set = set()
    for elem in common_list[:numb]:
        elem = elem.split()
        ind = int(elem[0]) - 51
        #pass_idx = doc_ind2pass[elem[2]]
        if ind not in ind_set:
            print (topics_2021[ind])
            ind_set.add(ind)
            print ("\n")
        print (big_dict[elem[2]])
        print ("\n")

## Colbert output

In [56]:
print_common_list(common_list, 10, {}, mapping_dict, big_dict)

('91', 'When are random forest classifiers better than decision trees?')


&quot;stochastic discrimination, random forests and boosting tend to form more accurate classifiers than decision trees alone.&quot;


; model a: 95% (96% correct), model b: 89% (92% correct), model c: 76% (97% correct)”. * provides a comprehensive set of machine learning algorithms from the weka project including clustering, segmentation, decision trees, random forests, neural networks, and principal component analysis. * pentaho has


using decision trees to construct a practical parser. kivinen and m. boosting as entropy projection. in proc. liu and x. ensemble learning via negative correlation. mason, p. bartlett, and j. direct optimization of margins improves generalization in combined classifiers.


these methods include: linear discriminant analysis, quadratic discriminant analysis, k-nearest neighbor classifier, bagging and boosting classification trees, support vector machine, and random forest


breima

## baseline output

In [279]:
print_common_list(common_list_bsln, 10, {}, mapping_dict, big_dict)

('51', 'What is better at reducing fever in children, Ibuprofen or Aspirin?')


additionally, these medications can become toxic, even at correct dosages, among young children suffering from dehydration due to fever, vomiting and&#x2f;or diarrhea. excess levels of acetaminophen or nsaids are not harmless.


but is your child&#x27;s fever really a cause for concern, and is lowering it the best choice? in most cases, a simple fever is actually a sign that your child&#x27;s immune system is working at its best.


in children, a fever that is equal to or greater than 102.2° should be treated. children older than two months of age with a fever of 102° or higher that does not respond to fever-reducing medication should be seen by a physician.


it is effective in reducing fever and is also prescribed for the treatment of arthritic conditions such rheumatoid- and osteoarthritis, muscle injury, and acute attacks of gout.


ibuprofen is a fast acting medicine and nurofen for children is an effe

In [57]:
with open('./colbert_91_100.qrels', 'w') as fp:
    fp.write('\n'.join(x for x in common_list))

In [51]:
common_list

[]

## Cut qrels for 91-100 queries

In [ ]:
qual_query

In [ ]:
tsv_file = open("/notebook/Touche22/touche2021_val/touche-task2-51-100-quality.qrels")
read_tsv = csv.reader(tsv_file, delimiter="\n")

common_list = []
i = 0

with open('/notebook/Touche22/touche2021_val/81_91_query/qual_81_91.qrels', 'w') as fp:
    for row in read_tsv:
        if (int(row[0].split()[0]) >= 80 and int(row[0].split()[0]) <90):
            fp.write(row[0])
            fp.write('\n')
            print(row)
            i = i +1

In [59]:
tsv_file = open("/notebook/Touche22/touche2021_val/outs/colbert.qrels")
read_tsv = csv.reader(tsv_file, delimiter="\n")

common_list = []
i = 0

with open('./colbert_ms_91_100.qrels', 'w') as fp:
    for row in read_tsv:
        if (int(row[0].split()[0]) > 90 and int(row[0].split()[0]) < 100):
            fp.write(row[0])
            fp.write('\n')
            if (i < 10):
                print(row)
                i = i +1

['91 Q0 clueweb12-0700wb-90-19409 1 1.0 colbert']
['91 Q0 clueweb12-0011wb-60-27834 2 0.5 colbert']
['91 Q0 clueweb12-1306wb-49-06042 3 0.3333333333333333 colbert']
['91 Q0 clueweb12-0812wb-61-14830 4 0.25 colbert']
['91 Q0 clueweb12-0810wb-00-06776 5 0.2 colbert']
['91 Q0 clueweb12-1006wb-58-12637 6 0.16666666666666666 colbert']
['91 Q0 clueweb12-1303wb-58-05620 7 0.14285714285714285 colbert']
['91 Q0 clueweb12-0500wb-08-20545 8 0.125 colbert']
['91 Q0 clueweb12-1413wb-94-04951 9 0.1111111111111111 colbert']
['91 Q0 clueweb12-0916wb-34-13384 10 0.1 colbert']


In [ ]:
Touche22/touche2021_val/outs/baseline.qrels

In [ ]:
'51 Q0 clueweb12-0001wb-15-05765 1 1984.913 MyBaselineFilterResponse\n'

In [35]:
common_list[:4]

['91 Q0 clueweb12-0700wb-90-19409 1 1.0 colbert',
 '91 Q0 clueweb12-1607wb-17-29270 2 0.5 colbert',
 '91 Q0 clueweb12-0809wb-99-02855 3 0.3333333333333333 colbert',
 '91 Q0 clueweb12-0309wb-14-10310 4 0.25 colbert']

In [232]:
common_list[:4]

['51 Q0 clueweb12-0905wb-28-28011 1 1.0 colbert\n',
 '51 Q0 clueweb12-0504wb-21-02022 2 0.5 colbert\n',
 '51 Q0 clueweb12-0010wb-17-25926 3 0.3333333333333333 colbert\n',
 '51 Q0 clueweb12-0003wb-21-31533 4 0.25 colbert\n']

In [ ]:
common_list = []

print ("baseline", flush = True)
for ind_q, elem in enumerate(list_of_tuples):
    qid = elem[0]
    Q0 = 'Q0'
    query = elem[1]
    tag = 'MyBaselineFilterResponse'
    responses = list(zip(*my_response_list[qid]))
        
    scores0 = responses[0]
    docs = responses[1]
    #print ("1")
    titles = responses[2]
    #print ("2")
    answers_bodies = responses[3]
    #print ("3")
    # print (scores0, scores3, scores)
    qids = [qid]*len(scores0)
    Q0s = [Q0 for elem in scores0]
    tags = [tag for elem in scores0]
    part_of_commom_list = list(zip(qids, Q0s, docs, scores0, tags))
    part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 

    qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)

    ranks = range(1, len(scores) + 1)
    part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
    common_list = common_list + part_of_commom_list
    print ("len(common_list)", len(common_list))

In [67]:
from transformers import AutoModel, AutoTokenizer

In [68]:
from transformers import BertModel
from transformers import BertPreTrainedModel
from transformers import BertConfig
import torch 
import torch.nn as nn

class VespaColBERT(BertPreTrainedModel):
   
    def __init__(self,config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.linear = nn.Linear(config.hidden_size, 32, bias=False)
        self.init_weights()
        
    def forward(self, input_ids, attention_mask):
        Q = self.bert(input_ids,attention_mask=attention_mask)[0]
        Q = self.linear(Q)
        return torch.nn.functional.normalize(Q, p=2, dim=2)  

colbert_query_encoder = VespaColBERT.from_pretrained("vespa-engine/colbert-medium") 

#Export model to ONNX for serving in Vespa 

input_names = ["input_ids", "attention_mask"]
output_names = ["contextual"]
#input, max 32 query term
input_ids = torch.ones(1,32, dtype=torch.int64)
attention_mask = torch.ones(1,32,dtype=torch.int64)
args = (input_ids, attention_mask)
torch.onnx.export(colbert_query_encoder,
                args=args,
                f="query_encoder_colbert.onnx",
                input_names = input_names,
                output_names = output_names,
                dynamic_axes = {
                    "input_ids": {0: "batch"},
                    "attention_mask": {0: "batch"},
                    "contextual": {0: "batch"},
                },
                opset_version=11)

Downloading:   0%|          | 0.00/541 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/158M [00:00<?, ?B/s]

In [72]:
model_name = "vespa-engine/colbert-medium" 


# Download pytorch model
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Transform input tokens 
inputs = tokenizer("Hello world!", return_tensors="pt")

# Model apply
outputs = model(**inputs)

Some weights of the model checkpoint at vespa-engine/colbert-medium were not used when initializing BertModel: ['linear.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [74]:
type(model)

transformers.models.bert.modeling_bert.BertModel

In [73]:
outputs.shape

AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'shape'

In [283]:
import requests
params = {
            "apikey": "0833a307-97d3-462a-99d9-27db400c70da",
            "query": 'When are random forest classifiers better than decision trees?',
            "index": ["cw12"],
            "size": 50,
            "pretty": True
        }
response = requests.get(url = "http://www.chatnoir.eu/api/v1/_search", params = params)

In [286]:
response.json()

{'error': {'code': 401, 'message': 'API key has expired'}}